In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nengo
import scipy
import time
from fit import *
from NEF_RL import *
from NEF_WM import *
palette = sns.color_palette('tab10')
sns.set_palette(palette)
sns.set(context='paper', style='white', font="cmr10", font_scale=1.2)
plt.rcParams['axes.formatter.use_mathtext'] = True

In [42]:
def run_and_get_noise_driven(model_type, sid, neurons, seed=0):
    # if model_type=='NEF_RL':
        # params = pd.read_pickle("data/NEF_RL_jiang_mar9_performance.pkl")
        # data = run_RL("jiang", sid, alpha=alpha, z=z, lambd=lambd)
        # math_data = pd.read_pickle("data/RL_z_jiang_mar7_dynamics.pkl").query("sid==@sid")
    rng = np.random.RandomState(seed=seed)
    if model_type=='NEF_WM':
        p_nef = pd.read_pickle("data/NEF_WM_jiang_mar11_params.pkl").query("sid==@sid")
        alpha = p_nef['alpha'].unique()[0]
        z = p_nef['z'].unique()[0]
        lambd = p_nef['lambda'].unique()[0]
        beta_nef = p_nef['beta'].unique()[0]
        params_nef = [z]
        nef_data = run_WM("jiang", sid, alpha=alpha, z=z, lambd=lambd, n_neurons=neurons, n_memory=neurons, n_error=neurons)
        math_type = "DG_z"
        p_math = pd.read_pickle(f"data/{math_type}_jiang_mar7_params.pkl").query("sid==@sid")
        z_math = p_math['z'].unique()[0]
        beta_math = p_math['beta'].unique()[0]
        params_math = [z_math, beta_math]
    human = pd.read_pickle("data/jiang.pkl").query("sid==@sid")
    trials = human.query("sid==@sid")['trial'].unique()
    columns = ['type', 'neurons', 'sid', 'trial', 'stage', 'is_greedy', 'noise_driven']
    dfs = []
    for trial in trials:
        # neighbors = len(human.query("sid==@sid & trial==@trial & stage==1")['who'].unique())
        stages = human.query("sid==@sid & trial==@trial")['stage'].unique()
        for stage in stages:
            expectation_math = get_expectations_jiang(math_type, params_math, sid, trial, stage)
            expectation_nef = get_expectations_jiang(model_type, params_nef, sid, trial, stage)
            prob_math = scipy.special.expit(beta_math*expectation_math)
            prob_nef = scipy.special.expit(beta_nef*expectation_nef)
            action_math = 1 if rng.uniform(0,1) < prob_math else -1
            action_nef = 1 if rng.uniform(0,1) < prob_nef else -1
            action_human = human.query("trial==@trial & stage==@stage")['action'].unique()[0]
            sign_math = 1 if expectation_math > 0 else -1
            sign_nef = 1 if expectation_nef > 0 else -1
            is_greedy = True if action_human==sign_math else False
            noise_driven = True if sign_nef!=sign_math else False
            dfs.append(pd.DataFrame([[model_type, neurons, sid, trial, stage, is_greedy, noise_driven]], columns=columns))
    noise_data = pd.concat(dfs, ignore_index=True)
    return noise_data

In [43]:
model_types = ['NEF_WM']
sids = pd.read_pickle("data/jiang.pkl")['sid'].unique()
neurons = [50, 100, 200, 400]
dfs = []
for mt in model_types:
    for sid in sids:
        for n in neurons:
            dfs.append(run_and_get_noise_driven(mt, sid=sid, neurons=n))
noise_data = pd.concat(dfs, ignore_index=True)
noise_data.to_pickle("data/jiang_noise_data.pkl")

sid 1, trial 1
sid 1, trial 2
sid 1, trial 3
sid 1, trial 4
sid 1, trial 5
sid 1, trial 6
sid 1, trial 7
sid 1, trial 8
sid 1, trial 9
sid 1, trial 10
sid 1, trial 11
sid 1, trial 12
sid 1, trial 13
sid 1, trial 14
sid 1, trial 15
sid 1, trial 16
sid 1, trial 17
sid 1, trial 18
sid 1, trial 19
sid 1, trial 20
sid 1, trial 21
sid 1, trial 22
sid 1, trial 23
sid 1, trial 24
sid 1, trial 25
sid 1, trial 26
sid 1, trial 27
sid 1, trial 28
sid 1, trial 29
sid 1, trial 30
sid 1, trial 31
sid 1, trial 32
sid 1, trial 33
sid 1, trial 34
sid 1, trial 35
sid 1, trial 36
sid 1, trial 37
sid 1, trial 38
sid 1, trial 39
sid 1, trial 40
sid 1, trial 1
sid 1, trial 2
sid 1, trial 3
sid 1, trial 4
sid 1, trial 5
sid 1, trial 6
sid 1, trial 7
sid 1, trial 8
sid 1, trial 9
sid 1, trial 10
sid 1, trial 11
sid 1, trial 12
sid 1, trial 13
sid 1, trial 14
sid 1, trial 15
sid 1, trial 16
sid 1, trial 17
sid 1, trial 18
sid 1, trial 19
sid 1, trial 20
sid 1, trial 21
sid 1, trial 22
sid 1, trial 23
sid 1, tri

KeyboardInterrupt: 

In [ ]:
dfs = []
columns = ['type', 'neurons', 'sid', 'is_greedy', 'noise_driven', "sample size"]
sid = noise_data['sid'].unique()
for model_type in noise_data['type'].unique():
    for sid in noise_data['sid'].unique():
        for neurons in noise_data['neurons'].unique():
            greedy_subdata = noise_data.query("type==@model_type & sid==@sid & neurons==@neurons & is_greedy==True")
            nongreedy_subdata = noise_data.query("type==@model_type & sid==@sid & neurons==@neurons & is_greedy==False")
            greedy_noisy = greedy_subdata.query("noise_driven==True").size
            greedy_choicy = greedy_subdata.query("noise_driven==False").size
            greedy_sample_size = greedy_noisy + greedy_choicy
            greedy_fraction = greedy_noisy / greedy_sample_size
            nongreedy_noisy = nongreedy_subdata.query("noise_driven==True").size
            nongreedy_choicy = nongreedy_subdata.query("noise_driven==False").size
            nongreedy_sample_size = nongreedy_noisy + nongreedy_choicy
            nongreedy_fraction = nongreedy_noisy / nongreedy_sample_size
            dfs.append(pd.DataFrame([[model_type, neurons, sid, True, greedy_fraction, greedy_sample_size]], columns=columns))
            dfs.append(pd.DataFrame([[model_type, neurons, sid, False, nongreedy_fraction, nongreedy_sample_size]], columns=columns))
noise_data_fraction = pd.concat(dfs, ignore_index=True)

In [ ]:
noise_data_fraction.to_pickle("data/noise_data_fraction.pkl")

In [ ]:
fig, ax = plt.subplots()
sns.kdeplot(noise_data_fraction.query("is_greedy==False"), x='noise_driven', hue='neurons',
            palette=palette[:6], common_norm=False, fill=True, clip=[0,1])
for n, neurons in enumerate(noise_data_fraction['neurons'].unique()):
    median = noise_data_fraction.query("is_greedy==False & neurons==@neurons")['noise_driven'].median()
    ax.axvline(median, color=palette[n], ls="--")
ax.set(xlabel="Fraction of Non-Greedy Choices Driven by Learning Noise")
plt.tight_layout()
fig.savefig("plots/learning_noise_vs_neurons_jiang.jpg", dpi=600)